In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras as k
import utils
%matplotlib inline

In [4]:
utils.FOI_COLUMNS

['FOI_hits_X',
 'FOI_hits_Y',
 'FOI_hits_T',
 'FOI_hits_Z',
 'FOI_hits_DX',
 'FOI_hits_DY',
 'FOI_hits_DT',
 'FOI_hits_DZ',
 'FOI_hits_S']

In [7]:
import gc
gc.collect()

0

In [6]:
# df = utils.load_train_hdf("data/")
df = pd.read_hdf("data/train_part_1_v2.hdf")

KeyboardInterrupt: 

In [ ]:
df.columns

In [ ]:
labels = df.label
df = df.loc[:, utils.FOI_COLUMNS]

In [ ]:
df.FOI_hits_DX

In [ ]:
max_hits = df.FOI_hits_S.apply(lambda x: np.unique(x, return_counts=True)[1])

In [ ]:
(max_hits.apply(lambda x: len(x)) == 4).mean()

In [ ]:
max_uncertainty.median()

In [ ]:
hits_per_station = max_hits.apply(lambda x: np.max(x))

In [ ]:
foi_hits = df.FOI_hits_DX.apply(lambda x: len(x))
hit_freqs = foi_hits.value_counts()
hit_freqs = hit_freqs[hit_freqs>3000]
used_freqs = set(hit_freqs.index)
print(np.sum(hit_freqs) / df.shape[0])
df = df[foi_hits.isin(used_freqs)]

## Loading keras model

In [ ]:
from keras.layers import Input, Conv2D, Dense, MaxPool2D, Masking,  AvgPool2D, Flatten, SeparableConv2D, GlobalAveragePooling2D, Concatenate, LSTM
from keras.models import Model
import keras.backend as K

In [ ]:
inputs = [Input(shape=(None, 6),) for _ in range(4)]
hs = [LSTM(32, activation="tanh")(inp) for inp in inputs]
h = Concatenate()(hs)
h = Dense(128, activation="relu")(h)
h = Dense(16, activation="relu")(h)
out = Dense(2, activation="sigmoid")(h)
model = Model(inputs, out)

## Process data into generators

In [ ]:
batch_size = 22
batches_used = dict(zip(hit_freqs.index, [0]*hit_freqs.shape[0])) # Dict batch_size -> number of batches taken overall
batch_counter = {} # Dict batch_size -> last batch processed
max_batches = dict(zip(hit_freqs.index, np.floor(hit_freqs.values / batch_size))) # Dict batch_size -> number of batches
batches = df.groupby(foi_hits).apply(lambda x: np.array(x)).to_dict() # Dict batch_size -> batches

def equilengthed_generator():
    while True:
        yield

In [ ]:
# TODO: assert keys are equal 
# TODO: split into val and train
# TODO: split each batch into [station 0, station 1, ...]

In [ ]:
# Nasty preprocessing! Kostil, since np.array is not working somehow to translate 1D array of 1D arrays into 2D array
for b_key, b_values in batches.items():
    new_batch = np.empty((len(b_values), 7, b_key))
    for i in range(b_values.shape[0]):
        for j in range(7):
            new_batch[i, j] = b_values[i, j]
    batches[b_key] = new_batch
    del b_values

In [ ]:
inp = Input((None, 7)) # overall hits * channels
station_masks = inp[:, 6] # for every hit check selection mask
inputs = [inp[station_masks == i, : 6] for i in range(4)]

In [ ]:
inp = Input(shape=(None, 7, None))
stations = inp[:,6]
inp = inp[:, :6]
masking = Masking()(inp)

l1 = LSTM(32, activation="tanh", return_state = True)
    
l2 = LSTM(32, activation="tanh", return_state = True)
l3 = LSTM(32, activation="tanh", return_state = True)
l4 = LSTM(32, activation="tanh")

# hs = [(inp) for inp in inputs]
# h = Concatenate()(hs)
# h = Dense(128, activation="relu")(h)
# h = Dense(16, activation="relu")(h)
# out = Dense(2, activation="sigmoid")(h)
# model = Model(inputs, out)

In [ ]:
h1